Raspando os personagens

In [36]:
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import matplotlib.pyplot as plt
import os
import logging

Estabelecendo o driver

In [37]:
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")

logging.getLogger('WDM').setLevel(logging.NOTSET)
os.environ['WDM_LOG'] = 'False'

webdriver_service = Service(ChromeDriverManager().install())

driver = webdriver.Chrome(service=webdriver_service,options=chrome_options)

Função para raspar personagens

In [38]:
def raspa(path):
    driver.get(path)
    categorias = driver.find_elements(by=By.CLASS_NAME,value='category-page__member-link')

    livros = []
    for categoria in categorias:
        livro_url = categoria.get_attribute('href')
        livro_name = categoria.text
        livros.append({'livro_name': livro_name, "url":livro_url})
    
    character_list = []
    for livro in livros:
        driver.get(livro['url'])
        character_elems = driver.find_elements(by=By.CLASS_NAME, value = 'category-page__member-link')

        for elem in character_elems:
            character_list.append({'livro': livro['livro_name'],'character': elem.text})
    return pd.DataFrame(character_list)

Romances

In [39]:
novels_path = "https://agathachristie.fandom.com/wiki/Category:Characters_by_novel"
romance = raspa(novels_path)
romance

,livro,character
0,Category:4.50 from Paddington characters,Lady Adington
1,Category:4.50 from Paddington characters,Inspector Awdry
2,Category:4.50 from Paddington characters,Inspector Bacon
3,Category:4.50 from Paddington characters,William Baker
4,Category:4.50 from Paddington characters,Miss Bartlett
...,...,...
1920,Category:Three Act Tragedy characters,Sir Bartholomew Strange
1921,Category:Three Act Tragedy characters,Angela Sutcliffe
1922,Category:Three Act Tragedy characters,Temple (Three Act Tragedy)
1923,Category:Three Act Tragedy characters,Alice West


Novelas

In [40]:
short_path = "https://agathachristie.fandom.com/wiki/Category:Characters_by_short_story"
novelas = raspa(short_path)


Peças

In [41]:
peca_path = "https://agathachristie.fandom.com/wiki/Category:Characters_by_stage_play"
pecas = raspa(peca_path)

Juntando as três em um dataframe

In [42]:
personagens = pd.merge(romance,novelas,how = 'outer')
personagens = pd.merge(personagens,pecas,how = 'outer')
personagens

,livro,character
0,Category:4.50 from Paddington characters,Alexander Eastley
1,Category:4.50 from Paddington characters,Alfred Crackenthorpe
2,Category:4.50 from Paddington characters,Alice Crackenthorpe
3,Category:4.50 from Paddington characters,Anna Stravinska
4,Category:4.50 from Paddington characters,Archibald Spate
...,...,...
2160,Category:Within a Wall characters,Miss Bates
2161,Category:Within a Wall characters,Mrs Lemprière
2162,Category:Within a Wall characters,Sir Rufus Herschman
2163,Category:Within a Wall characters,The Marches


In [43]:
#personagens = personagens.drop('livro', axis = 1)


In [44]:
personagens

,character
0,Alexander Eastley
1,Alfred Crackenthorpe
2,Alice Crackenthorpe
3,Anna Stravinska
4,Archibald Spate
...,...
2160,Miss Bates
2161,Mrs Lemprière
2162,Sir Rufus Herschman
2163,The Marches


In [45]:
personagens = personagens.drop_duplicates()

In [46]:
personagens

,character
0,Alexander Eastley
1,Alfred Crackenthorpe
2,Alice Crackenthorpe
3,Anna Stravinska
4,Archibald Spate
...,...
2160,Miss Bates
2161,Mrs Lemprière
2162,Sir Rufus Herschman
2163,The Marches


Gerando csv com os nomes dos personagens

In [47]:
personagens['character'].to_csv('personagens.csv',index=False,header=False)

Alguns personagens chave ficaram faltando, então acrescentei manualmente:

In [48]:
import io
arquivo_csv = 'personagens.csv'
arquivo_txt = 'faltando.txt'

try:
    df_testes = pd.read_csv(arquivo_csv)
except FileNotFoundError:
    print(f"Erro: O arquivo '{arquivo_csv}' não foi encontrado.")
    exit()

try:
    with open(arquivo_txt, 'r', encoding='utf-8') as f:
        linhas_faltando = f.read().splitlines()
except FileNotFoundError:
    print(f"Erro: O arquivo '{arquivo_txt}' não foi encontrado.")
    exit()

# 3. Criar um DataFrame a partir das linhas a serem adicionadas
# Assumindo que a coluna no CSV é a primeira (índice 0). Você pode precisar ajustar o nome da coluna.
nome_da_coluna = df_testes.columns[0] if not df_testes.empty else 'Nome' # Tenta pegar o nome da primeira coluna ou usa 'Nome'
df_faltando = pd.DataFrame(linhas_faltando, columns=[nome_da_coluna])

# 4. Acrescentar (concatenar) as novas linhas ao DataFrame original
# 'ignore_index=True' garante que o novo DataFrame tenha um índice sequencial correto
df_resultado = pd.concat([df_testes, df_faltando], ignore_index=True)

# 5. Salvar o DataFrame resultante de volta em um novo arquivo CSV (ou substituir o original, se desejar)
arquivo_saida = 'personagens.csv'
df_resultado.to_csv(arquivo_saida, index=False) # index=False evita escrever o índice do DataFrame como uma coluna no CSV
